In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
from quaternion import Quaternion

In [85]:
def scale_value(value, max_value, max_angle):
    return round((abs(value) / max_angle) * max_value)

def scale_roll(roll, scale):
    return scale_value(roll, scale, 180)

def scale_pitch(pitch, scale):
    return scale_value(pitch, scale, 90)
    
def calculate_expected_pitch(truth_value, scale):
    if scale == 49:
        return (truth_value / 49.0) * 90
    elif scale == 100:
        return (truth_value / 100.0) * 90
    
def calculate_expected_roll(truth_value, scale):
    if scale == 49:
        return (truth_value / 49.0) * 180
    elif scale == 100:
        return (truth_value / 100.0) * 180

stimuli = []
scale_type = {
    "gesture_pitch_number": 100,
    "gesture_pitch_greyscale": 49,
    "gesture_roll_number": 100,
    "gesture_roll_greyscale": 49,
    "slider_number": 1,
    "slider_greyscale": 1,
}

euler_type = {
    "gesture_pitch_number": "XYZ",
    "gesture_pitch_greyscale": "XYZ",
    "gesture_roll_number": "XYZ",
    "gesture_roll_greyscale": "XYZ",
    "slider_number": "XYZ",
    "slider_greyscale": "XYZ",
}

with open('experiment_data/experiment_data_14.json', 'r') as file:
    data = json.load(file)

    if data:
        for experiment in data["completedExperiments"]:
            experiment_type = experiment.get("experimentType")
            for stimulus in experiment["successfulStimuli"]:
                sensor_reading = stimulus.get("sensorReading", {})
                quaternion = Quaternion(
                    sensor_reading.get("w"),
                    sensor_reading.get("x"),
                    sensor_reading.get("y"),
                    sensor_reading.get("z")
                )
                if None not in quaternion:
                    yaw, pitch, roll = quaternion.euler_angles(euler_type[experiment_type])
                    
                    pitch_deg = math.degrees(pitch)
                    roll_deg = math.degrees(roll)
                    
                    sensor_reading["roll"] = roll_deg
                    sensor_reading["pitch"] = pitch_deg

                    pitch_scaled = scale_pitch(pitch_deg, scale_type[experiment_type])
                    roll_scaled = scale_roll(roll_deg, scale_type[experiment_type])
                    truth_value = stimulus.get("truth")
                    expected_pitch = calculate_expected_pitch(truth_value, scale_type[experiment_type])
                    expected_roll = calculate_expected_roll(truth_value, scale_type[experiment_type])

                    stimulus["pitch"] = pitch_deg
                    stimulus["expected_pitch"] = expected_pitch
                    stimulus["pitch_truth"] = pitch_scaled
                    stimulus["pitch_truth_diff"] = abs(truth_value - pitch_scaled)
                    stimulus["roll"] = roll_deg
                    stimulus["expected_roll"] = expected_roll
                    stimulus["roll_truth"] = roll_scaled
                    stimulus["roll_truth_diff"] = abs(truth_value - roll_scaled)
                    stimulus["experiment_type"] = experiment_type
                    
                    stimuli.append(stimulus)

df = pd.DataFrame(data=stimuli)

In [86]:
df[df["experiment_type"] == "gesture_pitch_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,expected_pitch,pitch_truth,pitch_truth_diff,roll,expected_roll,roll_truth,roll_truth_diff,experiment_type
0,"{'w': 0.00860596, 'z': 0.789124, 'stability': ...",device,0.0,20,1,-17.021319,36.734694,9,11,75.121226,73.469388,20,0,gesture_pitch_greyscale
1,"{'duration': 167, 'x': -0.204102, 'stability':...",device,0.0,5,2,-24.114059,9.183673,13,8,74.650535,18.367347,20,15,gesture_pitch_greyscale
2,"{'z': 0.855957, 'w': -0.0631714, 'stability': ...",device,0.0,48,3,-2.274431,88.163265,1,47,61.728857,176.326531,17,31,gesture_pitch_greyscale
3,"{'stability': 'In motion', 'z': 0.62384, 'cali...",device,0.0,29,4,-69.086773,53.265306,38,9,19.657833,106.530612,5,24,gesture_pitch_greyscale
4,"{'w': 0.171936, 'duration': 190, 'activity': '...",device,0.0,46,5,-38.093535,84.489796,21,25,75.616748,168.979592,21,25,gesture_pitch_greyscale
5,"{'y': -0.395081, 'stability': 'In motion', 'ac...",device,0.0,0,6,-72.654869,0.000000,40,40,-2.607994,0.000000,1,1,gesture_pitch_greyscale
6,"{'y': -0.501587, 'timestamp': 1719319643, 'x':...",device,0.0,28,7,-2.822161,51.428571,2,26,60.444552,102.857143,16,12,gesture_pitch_greyscale
7,"{'calibration_status': 2, 'x': -0.282837, 'y':...",device,0.0,37,8,-39.039612,67.959184,21,16,71.796841,135.918367,20,17,gesture_pitch_greyscale
8,"{'timestamp': 1719319653, 'y': -0.483765, 'dur...",device,0.0,43,9,-67.479649,78.979592,37,6,59.192943,157.959184,16,27,gesture_pitch_greyscale
9,"{'timestamp': 1719319658, 'calibration_status'...",device,0.0,3,10,-68.410203,5.510204,37,34,36.600186,11.020408,10,7,gesture_pitch_greyscale


In [87]:
df[df["experiment_type"] == "gesture_pitch_number"]

,sensorReading,inputType,value,truth,id,pitch,expected_pitch,pitch_truth,pitch_truth_diff,roll,expected_roll,roll_truth,roll_truth_diff,experiment_type
20,"{'w': 0.180237, 'x': -0.390442, 'calibration_s...",device,0.0,64,1,-50.355472,57.6,56,8,68.624440,115.2,38,26,gesture_pitch_number
21,"{'w': 0.224854, 'x': -0.380798, 'calibration_s...",device,0.0,67,2,-52.480780,60.3,58,9,63.882263,120.6,35,32,gesture_pitch_number
22,"{'calibration_status': 2, 'duration': 181, 'y'...",device,0.0,91,3,-66.324511,81.9,74,17,29.445251,163.8,16,75,gesture_pitch_number
23,"{'x': -0.392883, 'activity_confidence': 91, 'z...",device,0.0,62,4,-52.460022,55.8,58,4,68.273340,111.6,38,24,gesture_pitch_number
24,"{'z': 0.732666, 'timestamp': 1719319747, 'acti...",device,0.0,74,5,-62.258151,66.6,69,5,61.964358,133.2,34,40,gesture_pitch_number
25,"{'duration': 190, 'activity_confidence': 94, '...",device,0.0,68,6,-53.104218,61.2,59,9,73.196819,122.4,41,27,gesture_pitch_number
26,"{'calibration_status': 2, 'y': -0.493164, 'tim...",device,0.0,28,7,-35.276182,25.2,39,11,69.632286,50.4,39,11,gesture_pitch_number
27,"{'timestamp': 1719319768, 'activity_confidence...",device,0.0,56,8,-40.787907,50.4,45,11,101.333621,100.8,56,0,gesture_pitch_number
28,"{'z': 0.651733, 'calibration_status': 2, 'dura...",device,0.0,57,9,-43.030687,51.3,48,9,99.950413,102.6,56,1,gesture_pitch_number
29,"{'duration': 154, 'y': -0.540833, 'x': -0.5241...",device,0.0,93,10,-65.869336,83.7,73,20,109.193458,167.4,61,32,gesture_pitch_number


In [88]:
df[df["experiment_type"] == "gesture_roll_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,expected_pitch,pitch_truth,pitch_truth_diff,roll,expected_roll,roll_truth,roll_truth_diff,experiment_type
40,"{'z': 0.689941, 'x': -0.481995, 'y': -0.472534...",device,0.0,44,1,-24.712011,80.816327,13,31,84.393331,161.632653,23,21,gesture_roll_greyscale
41,"{'activity_confidence': 73, 'y': -0.430176, 'c...",device,0.0,23,2,-28.369418,42.244898,15,8,88.237162,84.489796,24,1,gesture_roll_greyscale
42,"{'duration': 191, 'w': -0.209595, 'activity_co...",device,0.0,36,3,-19.038063,66.122449,10,26,74.033314,132.244898,20,16,gesture_roll_greyscale
43,"{'duration': 154, 'activity_confidence': 83, '...",device,0.0,9,4,-20.127634,16.530612,11,2,75.132017,33.061224,20,11,gesture_roll_greyscale
44,"{'duration': 191, 'timestamp': 1719319897, 'w'...",device,0.0,25,5,-11.535929,45.918367,6,19,70.064513,91.836735,19,6,gesture_roll_greyscale
45,"{'calibration_status': 1, 'duration': 154, 'st...",device,0.0,32,6,-15.319396,58.775510,8,24,71.315636,117.551020,19,13,gesture_roll_greyscale
46,"{'y': 0.219299, 'timestamp': 1719319938, 'stab...",device,0.0,49,7,-49.714035,90.000000,27,22,-50.784905,180.000000,14,35,gesture_roll_greyscale
47,"{'y': -0.263794, 'z': 0.584167, 'calibration_s...",device,0.0,4,8,-40.744331,7.346939,22,18,99.142009,14.693878,27,23,gesture_roll_greyscale
48,"{'timestamp': 1719319973, 'calibration_status'...",device,0.0,6,9,-13.868960,11.020408,8,2,57.334172,22.040816,16,10,gesture_roll_greyscale
49,"{'duration': 208, 'w': -0.141602, 'x': -0.2551...",device,0.0,24,10,-18.085294,44.081633,10,14,59.105641,88.163265,16,8,gesture_roll_greyscale


In [89]:
df[df["experiment_type"] == "gesture_roll_number"]

,sensorReading,inputType,value,truth,id,pitch,expected_pitch,pitch_truth,pitch_truth_diff,roll,expected_roll,roll_truth,roll_truth_diff,experiment_type
60,"{'duration': 213, 'activity_confidence': 95, '...",device,0.0,75,1,-29.161936,67.5,32,43,71.966307,135.0,40,35,gesture_roll_number
61,"{'activity_confidence': 96, 'w': -0.284912, 's...",device,0.0,81,2,-30.306047,72.9,34,47,75.050443,145.8,42,39,gesture_roll_number
62,"{'z': 0.770508, 'stability': 'Stable', 'y': -0...",device,0.0,70,3,-26.907048,63.0,30,40,69.230267,126.0,38,32,gesture_roll_number
63,"{'activity': 'Still', 'calibration_status': 2,...",device,0.0,26,4,-16.443609,23.4,18,8,60.701576,46.8,34,8,gesture_roll_number
64,"{'calibration_status': 2, 'activity': 'Still',...",device,0.0,20,5,-14.974565,18.0,17,3,60.632365,36.0,34,14,gesture_roll_number
65,"{'calibration_status': 2, 'y': -0.397583, 'tim...",device,0.0,63,6,-26.460857,56.7,29,34,67.817110,113.4,38,25,gesture_roll_number
66,"{'z': 0.73645, 'stability': 'Stable', 'timesta...",device,0.0,85,7,-28.833356,76.5,32,53,73.545123,153.0,41,44,gesture_roll_number
67,"{'activity_confidence': 80, 'activity': 'Still...",device,0.0,100,8,-36.953349,90.0,41,59,99.318782,180.0,55,45,gesture_roll_number
68,"{'activity': 'Still', 'stability': 'Stable', '...",device,0.0,96,9,-36.443078,86.4,40,56,92.440507,172.8,51,45,gesture_roll_number
69,"{'calibration_status': 2, 'duration': 231, 'x'...",device,0.0,46,10,-28.612240,41.4,32,14,66.846747,82.8,37,9,gesture_roll_number
